In [ ]:
import lightgbm as lgb
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

from google.colab import drive  
from lightgbm.sklearn import LGBMClassifier
from scipy import stats
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
drive.mount("/content/drive") 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/drive/My Drive/ks-projects-201801.csv")

In [ ]:
print(data.info())
data["category"].value_counts().plot.barh(figsize=(25,35))

In [ ]:
# Let's get an idea as to the distribution of different outcomes
data["state"].value_counts().plot.bar()

In [ ]:
data.head()
categorical_features = ["category", "main_category", "country"]

numeric_features = ["usd_pledged_real", "usd_goal_real", "backers"]

categorical_transformer = Pipeline(steps=[
                ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
                ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

numeric_transformer = Pipeline(steps=[
                ("imputer", SimpleImputer(fill_value="median")),
                ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
          ("num", numeric_transformer, numeric_features),
          ("cat", categorical_transformer, categorical_features)
    ]
)

# Apply relevant pipeline operations to each of the column types
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


Model


In [ ]:
# First, let's save the categorical features
data.head()
categorical_features = ["category", "main_category", "country"]

numeric_features = ["usd_pledged_real", "usd_goal_real", "backers"]

categorical_transformer = Pipeline(steps=[
                ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
                ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

numeric_transformer = Pipeline(steps=[
                ("imputer", SimpleImputer(fill_value="median")),
                ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
          ("num", numeric_transformer, numeric_features),
          ("cat", categorical_transformer, categorical_features)
    ]
)

# Apply relevant pipeline operations to each of the column types
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
# Let's make the target attribute a bit prettier, and split the data into train/test
# Drop live projects
data = data.query('state != "live" and state != "undefined"')

# Add outcome column, "successful" == 1, others are 0
data = data.assign(outcome=(data['state'] == 'successful').astype(int))

In [ ]:
# data = data.drop(["state", "ID", "name", "currency", "deadline", "goal", "launched","pledged", "usd pledged"], axis=1)

y = data.outcome
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3)

# Select features from sets to be used
X_train = X_train[[*numeric_features, *categorical_features]]
X_test = X_test[[*numeric_features, *categorical_features]]

# X_train.drop(["outcome"], axis=1)
# for each in [X_train, X_test]:
#     print(f"Outcome fraction = {each.outcome.mean():.4f}")

In [ ]:
# Apply the transformers to the columns of the dataset
preprocessor = ColumnTransformer(
    transformers=[
          ("num", numeric_transformer, numeric_features),
          ("cat", categorical_transformer, categorical_features)
    ]
)

# Apply relevant pipeline operations to each of the column types
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

X_train = data.iloc["usd_pledged_real"]
lir = LinearRegression()
lir.fit()



NameError: ignored

In [ ]:
# Apply the transformers to the columns of the dataset
preprocessor = ColumnTransformer(
    transformers=[
          ("num", numeric_transformer, numeric_features),
          ("cat", categorical_transformer, categorical_features)
    ]
)

# Apply relevant pipeline operations to each of the column types
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Build classification pipeline for LightGBM
regr = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', LGBMClassifier(boosting_type='gbdt',
                        objective='binary'
                      ))])


# Select features from sets to be used
# X_test.drop(["outcome"], axis=1)

# `__` denotes attribute 
# (e.g. regressor__n_estimators means the `n_estimators` param for `regressor`
#  which is our xgb).
param_grid = {
    'preprocessor__num__imputer__strategy': ['median'],
    'regressor__num_leaves': [30, 45],
    'regressor__n_estimators': [200, 300],
    'regressor__max_depth': [10, 15],
    'regressor__learning_rate': [0.03, 0.05],
    'regressor__min_data_in_leaf' : [10, 20],
}

# Grid search the parameters above, 3-fold cv
grid_search = GridSearchCV(
    regr, param_grid, cv=3, verbose=3, n_jobs=1, 
    scoring='accuracy')

# Parameter optimization and model fitting
grid_search.fit(X_train, y_train)

In [ ]:
y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
y_pred = pd.Series(y_pred)

In [ ]:
# this line crashes it because it runs out of ram... not sure what to do about that
#finished = X_test.append(y_pred, ignore_index=True)

In [ ]:
X_train